In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

df = pd.read_csv('../data/csgo_round_snapshots.csv')
X_train = pd.read_csv('../data/processed/X_train.csv')
X_test = pd.read_csv('../data/processed/X_test.csv')
y_train_reg = pd.read_csv('../data/processed/y_train_reg_scaled.csv')
y_test_reg = pd.read_csv('../data/processed/y_test_reg_scaled.csv')

In [5]:
n_col = X_train.shape[1]
model_regressor = tf.keras.Sequential([
    tf.keras.Input(shape=(n_col,)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(16, activation='relu'),

    tf.keras.layers.Dense(2, activation='linear')
])

model_regressor.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        loss='mse',
                        metrics=['mae'])

start_training = time.time()
history = model_regressor.fit(
    X_train, y_train_reg,
    validation_split=0.2,
    epochs=1,
    batch_size=64,
    verbose=1
)
end_training = time.time() - start_training
print(f"Temps d'entraînement : {end_training:.2f} secondes")

1225/1225 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.3972 - mae: 0.3820 - val_loss: 0.2374 - val_mae: 0.2749
Temps d'entraînement : 6.97 secondes


In [6]:
df_raw = pd.read_csv('../data/csgo_round_snapshots.csv')
y_reg_raw = df_raw[['ct_money', 'ct_health']]


_, _, y_train_real, y_test_real = train_test_split(
    df_raw, y_reg_raw, test_size=0.2, random_state=42
)


scaler_correct = StandardScaler()
scaler_correct.fit(y_train_real)

y_pred_train_scaled = model_regressor.predict(X_train, verbose=0)

start_pred = time.time()
y_pred_test_scaled = model_regressor.predict(X_test, verbose=0)
end_pred = time.time() - start_pred

y_pred_train_final = scaler_correct.inverse_transform(y_pred_train_scaled)
y_pred_test_final = scaler_correct.inverse_transform(y_pred_test_scaled)

train_rmse = np.sqrt(mean_squared_error(y_train_real, y_pred_train_final))
test_rmse = np.sqrt(mean_squared_error(y_test_real, y_pred_test_final))

train_mae = mean_absolute_error(y_train_real, y_pred_train_final)
test_mae = mean_absolute_error(y_test_real, y_pred_test_final)

print(f"Temps de prédiction : {end_pred:.4f} s")
print(f"\nGlobal Train RMSE : {train_rmse:.2f}")
print(f"Global Test RMSE  : {test_rmse:.2f}")

mae_per_col = mean_absolute_error(y_test_real, y_pred_test_final, multioutput='raw_values')

print(f"\nErreur Argent : {mae_per_col[0]:.0f} dollars")
print(f"Erreur Santé  : {mae_per_col[1]:.0f} HP")

Temps de prédiction : 1.3908 s

Global Train RMSE : 4380.97
Global Test RMSE  : 4363.78

Erreur Argent : 4211 dollars
Erreur Santé  : 22 HP


In [2]:
import optuna

def objective(trial):
    activation_chosen = trial.suggest_categorical('activation', ['relu', 'elu', 'swish'])
    units_1 = trial.suggest_int('units_1', 16, 100)
    units_2 = trial.suggest_int('units_2', 16, 100)
    dropout_rate_1 = trial.suggest_float('dropout_rate_1', 0.0, 0.5)
    dropout_rate_2 = trial.suggest_float('dropout_rate_2', 0.1, 0.5)
    l2_reg = trial.suggest_float('l2_reg', 0.00001, 0.01, log=True)
    learning_rate = trial.suggest_float('learning_rate', 0.0001, 0.01, log=True)

    n_col = X_train.shape[1]
    model_regressor_opti = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(n_col,)),
        tf.keras.layers.Dense(units_1, activation=activation_chosen, kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        tf.keras.layers.Dropout(dropout_rate_1),

        tf.keras.layers.Dense(units_2, activation=activation_chosen, kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        tf.keras.layers.Dropout(dropout_rate_2),

        tf.keras.layers.Dense(2, activation='linear')
    ])
    model_regressor_opti.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    history = model_regressor_opti.fit(
        X_train, y_train_reg,
        validation_split=0.2,
        epochs=20,
        batch_size=64,
        verbose=0
    )
    return history.history['val_loss'][-1]

storage_name = "sqlite:///../db.sqlite3"
study = optuna.create_study(
    study_name="optimization_csgo",
    storage=storage_name,
    direction='minimize',
    load_if_exists=True
)
study.optimize(objective, n_trials=30)
print(f"Best loss :{study.best_value}")
print(f"Meilleur hyperparametres: {study.best_params}")

C:\Users\jevsez\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-12-17 14:49:48,091] A new study created in RDB with name: optimization_csgo
[I 2025-12-17 14:51:14,123] Trial 0 finished with value: 0.1256505399942398 and parameters: {'activation': 'relu', 'units_1': 62, 'units_2': 98, 'dropout_rate_1': 0.017805387202671163, 'dropout_rate_2': 0.4384202386690238, 'l2_reg': 1.4399985977344455e-05, 'learning_rate': 0.0007548957291935124}. Best is trial 0 with value: 0.1256505399942398.
[I 2025-12-17 14:52:34,673] Trial 1 finished with value: 0.16300678253173828 and parameters: {'activation': 'relu', 'units_1': 97, 'units_2': 82, 'dropout_rate_1': 0.32869712217148694, 'dropout_rate_2': 0.2837203151943375, 'l2_reg': 0.004587154023912081, 'learning_rate': 0.000379920330622993}. Best is

Best loss :0.12316789478063583
Meilleur hyperparametres: {'activation': 'swish', 'units_1': 80, 'units_2': 92, 'dropout_rate_1': 0.0083824542541171, 'dropout_rate_2': 0.19051185986270774, 'l2_reg': 2.3897177743862053e-05, 'learning_rate': 0.0006364951406449315}


In [ ]:
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

best_params_reg = study.best_params
print(f"Paramètres gagnants : {best_params_reg}")

n_col = X_train.shape[1]
model_reg_final = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_col,)),

    tf.keras.layers.Dense(best_params_reg['units_1'],
                          activation=best_params_reg['activation'],
                          kernel_regularizer=tf.keras.regularizers.l2(best_params_reg['l2_reg'])),
    tf.keras.layers.Dropout(best_params_reg['dropout_rate_1']),

    tf.keras.layers.Dense(best_params_reg['units_2'],
                          activation=best_params_reg['activation'],
                          kernel_regularizer=tf.keras.regularizers.l2(best_params_reg['l2_reg'])),
    tf.keras.layers.Dropout(best_params_reg['dropout_rate_2']),

    tf.keras.layers.Dense(2, activation='linear')   
])

model_reg_final.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=best_params_reg['learning_rate']),
    loss='mse',
    metrics=['mae']
)

history = model_reg_final.fit(
    X_train, y_train_reg,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    verbose=1
)

In [15]:
y_pred_scaled = model_reg_final.predict(X_test)

y_test_real = scaler_correct.inverse_transform(y_test_reg)
y_pred_real = scaler_correct.inverse_transform(y_pred_scaled)

r2 = r2_score(y_test_real, y_pred_real, multioutput='raw_values')
mae_per_column = mean_absolute_error(y_test_real, y_pred_real, multioutput='raw_values')

print(f"argent : Erreur moyenne: {mae_per_column[0]:.2f}")
print(f"PV : Erreur moyenne: {mae_per_column[1]:.2f}")
print(f"r2 argent : {r2[0]:.2f}")
print(f"r2 PV : {r2[1]:.2f}")

766/766 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
argent : Erreur moyenne: 2872.93
PV : Erreur moyenne: 15.83
r2 argent : 0.84
r2 PV : 0.96
